In [1]:
from datasets import load_dataset

# 加载 tinyMMLU 数据集
tiny_data = load_dataset('tinyBenchmarks/tinyMMLU')['test']


In [2]:
# 打印示例
#print(tiny_data.shape)
#print(tiny_data[0])

In [6]:
from zhipuai import ZhipuAI
import re
import os
import json

# 初始化模型客户端
client = ZhipuAI(api_key="……")  # API Key
# 定义正则表达式用于提取答案
answer_pattern = r"Answer:?\s*([A-D])"

# 存储响应数据的文件路径
response_file = "responses.json"

# 如果文件存在，则加载数据
if os.path.exists(response_file):
    with open(response_file, "r", encoding="utf-8") as f:
        response_data = json.load(f)
else:
    response_data = {}  # 初始化为空字典


In [7]:
# 遍历数据集
for idx, sample in enumerate(tiny_data):
    prompt = sample["input_formatted"]

    # 如果当前问题已存在于响应数据中，跳过调用
    if str(idx) in response_data:
        continue

    # 调用模型 API
    response = client.chat.completions.create(
        model="glm-4-9b",
        messages=[{"role": "user", "content": prompt}],
        stream=False
    )

    # 获取完整生成的文本
    generated_text = response.choices[0].message.content.strip()

    # 保存到字典中，包含问题下标
    response_data[str(idx)] = {
        "index": idx,  # 问题下标
        "question": sample["question"],
        "choices": sample["choices"],
        "correct_answer": sample["choices"][sample["answer"]],
        "generated_response": generated_text
    }

# 提示处理完成
print("模型生成的响应已处理完成。")


模型生成的响应已处理完成。


In [8]:
# 将响应字典写入 JSON 文件
with open(response_file, "w", encoding="utf-8") as f:
    json.dump(response_data, f, ensure_ascii=False, indent=4)

print("所有响应已保存至 responses_with_index.json 文件。")


所有响应已保存至 responses_with_index.json 文件。


In [9]:
import pandas as pd
import numpy as np

file_path = "answer.xlsx"
# 读取文件，将第一列设置为索引，并指定没有表头
df = pd.read_excel(file_path, header=None, index_col=0)
# 将第二列重命名为 'Answer' 或其他你喜欢的名称
df.columns = ['Predicted Answer']
# 查看数据
print(df)

   Predicted Answer
0                  
0                 D
1                 C
2                 B
3                 F
4                 C
..              ...
95                A
96                A
97                B
98                A
99                F

[100 rows x 1 columns]


In [10]:
# 初始化评分向量
y = []

# 定义选项映射表
answer_map = {"A": 0, "B": 1, "C": 2, "D": 3}
num = 0
# 遍历 tinyMMLU 数据集
for idx, sample in enumerate(tiny_data):
    # 检查当前索引是否在预测数据中
    if idx in df.index:
        # 获取预测答案
        predicted_answer = df.loc[idx, "Predicted Answer"].strip()
        #print(predicted_answer)
        predicted_answer_mapped = answer_map.get(predicted_answer, "-1")  # 映射为数字，若无效返回 "-1"
        #print(predicted_answer_mapped)
        # 获取正确答案
        correct_index = sample["answer"]  # 正确答案索引
        correct_answer = sample["choices"][correct_index]  # 正确答案
        
        #print(correct_index)

        # 比较预测答案与正确答案
        if predicted_answer_mapped == correct_index:
            y.append(1)  # 正确
            num= num +1
        else:
            y.append(0)  # 错误

# 转换为 NumPy 数组
y = np.array(y)

# 打印评分向量
print("评分向量:", y)
print("正确率",num/100)

评分向量: [1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1
 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0]
正确率 0.69


In [11]:
import tinyBenchmarks as tb

# 指定评估基准为 MMLU
benchmark = 'mmlu'

# 计算模型的评估分数
results = tb.evaluate(y, benchmark)

# 输出评估结果
print(results)


{'mmlu': {'irt': 0.7186072378115177, 'pirt': 0.6456080154503729, 'gpirt': 0.6537071224211837}}
